# Microplan Excel Validator

Run all cells (Shift+Enter) to start.

In [1]:
# Setup and imports
import subprocess, sys, os, shutil, importlib

if os.path.exists('requirements.txt'):
    subprocess.run([sys.executable, '-m', 'pip', 'install', '-q', '-r', 'requirements.txt'], capture_output=True)

import pandas as pd
import ipywidgets as widgets
from IPython.display import display, HTML, clear_output
from collections import defaultdict
import base64

import validator as validator_module
importlib.reload(validator_module)
from validator import Validator

for folder in ['uploads', 'error']:
    if os.path.exists(folder):
        shutil.rmtree(folder)
    os.makedirs(folder, exist_ok=True)

validator = Validator()
print("Ready!")

Ready!


In [2]:
# Column Configuration

config_state = {
    'level_columns': [], 'target_columns': [], 'num_targets': 0,
    'facility_col': '', 'district_col': '', 'state_col': '',
    'alignment_mapping': {}, 'configured': False
}

level_boxes = []
target_boxes = []
out_config = widgets.Output()

# Boundary config widgets
first_level = widgets.Text(value='', placeholder='e.g., COUNTRY', description='Level 1:',
                           style={'description_width': '80px'}, layout=widgets.Layout(width='400px'))
level_boxes.append(first_level)
level_container = widgets.VBox([first_level])

btn_add_level = widgets.Button(description='+ Add Level', button_style='info', layout=widgets.Layout(width='120px'))

def add_level(btn):
    n = len(level_boxes) + 1
    box = widgets.Text(value='', placeholder='e.g., District', description=f'Level {n}:',
                       style={'description_width': '80px'}, layout=widgets.Layout(width='400px'))
    level_boxes.append(box)
    level_container.children = list(level_boxes)

btn_add_level.on_click(add_level)

num_targets = widgets.IntText(value=0, description='# Targets:', style={'description_width': '80px'},
                              layout=widgets.Layout(width='200px'))
target_container = widgets.VBox([])

def update_targets(change):
    target_boxes.clear()
    for i in range(change['new']):
        box = widgets.Text(value=f'target_{i+1}', description=f'Target {i+1}:',
                           style={'description_width': '80px'}, layout=widgets.Layout(width='400px'))
        target_boxes.append(box)
    target_container.children = target_boxes

num_targets.observe(update_targets, names='value')

# Facility config with mapping
facility_col = widgets.Text(value='Facility Name', description='Facility:', 
                            style={'description_width': '100px'}, layout=widgets.Layout(width='280px'))
facility_map = widgets.Text(value='Unidade Sanitaria', description='Maps to:',
                            style={'description_width': '70px'}, layout=widgets.Layout(width='220px'))

district_col = widgets.Text(value='District', description='District:',
                            style={'description_width': '100px'}, layout=widgets.Layout(width='280px'))
district_map = widgets.Text(value='Distrito', description='Maps to:',
                            style={'description_width': '70px'}, layout=widgets.Layout(width='220px'))

state_col = widgets.Text(value='State', description='State:',
                         style={'description_width': '100px'}, layout=widgets.Layout(width='280px'))
state_map = widgets.Text(value='Provincia', description='Maps to:',
                         style={'description_width': '70px'}, layout=widgets.Layout(width='220px'))

btn_save = widgets.Button(description='Save Config', button_style='success', layout=widgets.Layout(width='150px'))

def save_config(btn):
    with out_config:
        clear_output(wait=True)
        levels = [b.value.strip() for b in level_boxes if b.value.strip()]
        targets = [b.value.strip() for b in target_boxes if b.value.strip()]
        
        if not levels:
            display(HTML("<p style='color:red'>Enter at least one level!</p>"))
            return
        
        mapping = {}
        if facility_col.value.strip() and facility_map.value.strip():
            mapping[facility_col.value.strip()] = facility_map.value.strip()
        if district_col.value.strip() and district_map.value.strip():
            mapping[district_col.value.strip()] = district_map.value.strip()
        if state_col.value.strip() and state_map.value.strip():
            mapping[state_col.value.strip()] = state_map.value.strip()
        
        config_state.update({
            'level_columns': levels, 'target_columns': targets, 'num_targets': len(targets),
            'facility_col': facility_col.value.strip(), 'district_col': district_col.value.strip(),
            'state_col': state_col.value.strip(), 'alignment_mapping': mapping, 'configured': True
        })
        
        validator.set_columns(boundary_cols=levels, facility_cols=[facility_col.value.strip()] if facility_col.value.strip() else [],
                              target_cols=targets, num_targets=len(targets))
        
        map_str = ', '.join([f'{k}→{v}' for k,v in mapping.items()])
        display(HTML(f"""<div style='padding:10px; background:#d4edda; border-radius:5px;'>
            <b>Saved!</b><br>Levels: {', '.join(levels)}<br>Targets: {', '.join(targets) or 'none'}
            <br>Mapping: {map_str or 'none'}</div>"""))

btn_save.on_click(save_config)

display(widgets.VBox([
    widgets.HTML("<h3>Boundary File</h3>"),
    level_container, btn_add_level,
    widgets.HTML("<b>Targets:</b>"), num_targets, target_container,
    widgets.HTML("<h3>Facility File (with mapping to boundary)</h3>"),
    widgets.HBox([facility_col, facility_map]),
    widgets.HBox([district_col, district_map]),
    widgets.HBox([state_col, state_map]),
    btn_save, out_config
]))

In [3]:
# File Upload & Validation

file_state = {'boundary_file': None, 'facility_file': None}
out_status = widgets.Output()
out_results = widgets.Output()
out_downloads = widgets.Output()


upload1 = widgets.FileUpload(accept='.xlsx,.xls,.csv', multiple=False)
upload2 = widgets.FileUpload(accept='.xlsx,.xls,.csv', multiple=False)

btn_validate = widgets.Button(description='VALIDATE', button_style='primary', layout=widgets.Layout(width='120px'))
btn_clear = widgets.Button(description='Clear', button_style='warning', layout=widgets.Layout(width='80px'))

def show_status(msg, color='black'):
    with out_status:
        clear_output(wait=True)
        display(HTML(f'<p style="color:{color}; font-weight:bold">{msg}</p>'))

def save_upload(uploader, key):
    if not uploader.value: return
    files = uploader.value
    info = files[0] if isinstance(files, tuple) else list(files.values())[0]
    name = info.name if hasattr(info, 'name') else info['name']
    content = info.content if hasattr(info, 'content') else info['content']
    path = os.path.join('uploads', name)
    with open(path, 'wb') as f: f.write(content)
    file_state[key] = path
    show_status(f'Loaded: {name}', 'green')

def on_validate(b):
    if not config_state['configured']:
        show_status('Configure columns first!', 'red')
        return
    if not file_state['boundary_file']:
        show_status('Upload boundary file!', 'red')
        return
    if not file_state['facility_file']:
        show_status('Upload facility file!', 'red')
        return
    
    validator.reset()
    show_status('Validating...', 'blue')
    all_issues = []
    summary = {'total': 0, 'errors': 0, 'warnings': 0, 'by_rule': defaultdict(int)}
    
    # Validate boundary file
    validator.set_columns(boundary_cols=config_state['level_columns'],
                          facility_cols=[config_state['facility_col']] if config_state['facility_col'] else [],
                          target_cols=config_state['target_columns'], num_targets=config_state['num_targets'])
    
    issues, s = validator.validate_file(file_state['boundary_file'])
    all_issues.extend(issues)
    summary['total'] += s['total']; summary['errors'] += s['errors']; summary['warnings'] += s['warnings']
    for r, c in s['by_rule'].items(): summary['by_rule'][r] += c
    
    # Get boundary data for alignment
    b_sheets = validator.read_file(file_state['boundary_file'])
    b_sheet = list(b_sheets.keys())[0]
    b_df = b_sheets[b_sheet]
    
    # Validate facility file
    fac_cols = [c for c in [config_state['facility_col'], config_state['district_col'], config_state['state_col']] if c]
    validator.set_columns(boundary_cols=fac_cols,
                          facility_cols=[config_state['facility_col']] if config_state['facility_col'] else [],
                          target_cols=[], num_targets=0)
    validator.set_alignment_mapping(config_state['alignment_mapping'])
    
    issues2, s2 = validator.validate_file(file_state['facility_file'])
    all_issues.extend(issues2)
    summary['total'] += s2['total']; summary['errors'] += s2['errors']; summary['warnings'] += s2['warnings']
    for r, c in s2['by_rule'].items(): summary['by_rule'][r] += c
    
    # Run alignment check
    if config_state['alignment_mapping']:
        f_sheets = validator.read_file(file_state['facility_file'])
        f_sheet = list(f_sheets.keys())[0]
        f_df = f_sheets[f_sheet]
        f_label = os.path.basename(file_state['facility_file'])
        
        if f_label not in validator.row_status:
            validator.init_row_status(f_df, f_label)
        
        align_issues = validator.check_alignment(b_df, f_df, b_sheet, f_label)
        all_issues.extend(align_issues)
        summary['total'] += len(align_issues)
        summary['errors'] += len([i for i in align_issues if i['severity'] == 'error'])
        for i in align_issues: summary['by_rule'][i['rule']] += 1
        
        if file_state['facility_file'] in validator.file_data:
            for sn, df in validator.file_data[file_state['facility_file']].items():
                if f_label in validator.row_status:
                    for idx, info in validator.row_status[f_label].items():
                        if idx in df.index:
                            df.loc[idx, 'VALIDATION_STATUS'] = info['status']
                            df.loc[idx, 'VALIDATION_ERRORS'] = '; '.join(info['errors'])
    
    output_files = validator.save_validated_files('error')
    display_results(all_issues, summary)
    display_downloads(output_files)
    show_status('Done!', 'green')

def on_clear(b):
    file_state['boundary_file'] = None
    file_state['facility_file'] = None
    validator.reset()
    with out_results: clear_output()
    with out_downloads: clear_output()
    show_status('Cleared', 'orange')

def display_results(issues, summary):
    with out_results:
        clear_output(wait=True)
        p, f = validator.get_stats()
        color = '#27ae60' if summary['errors'] == 0 else '#e74c3c'
        
        html = f'''<div style="padding:10px; background:#f0f0f0; border-left:4px solid {color}; margin:10px 0;">
            <b style="color:{color}">{'All Passed!' if summary['errors']==0 else 'Issues Found'}</b>
            | PASS: {p} | FAIL: {f} | Warnings: {summary['warnings']}</div>'''
        
        if summary['by_rule']:
            html += '<b>By Rule:</b> ' + ', '.join([f'{r}: {c}' for r,c in summary['by_rule'].items()])
        
        if issues:
            html += '<div style="max-height:250px; overflow-y:auto; margin-top:10px;">'
            html += '<table style="width:100%; font-size:11px; border-collapse:collapse;">'
            html += '<tr style="background:#333; color:white;"><th>Sev</th><th>Rule</th><th>Sheet</th><th>Col</th><th>Row</th><th>Value</th><th>Message</th></tr>'
            for i in issues[:50]:
                c = '#c00' if i['severity']=='error' else '#d80'
                html += f'<tr><td style="color:{c}">{i["severity"][:3].upper()}</td><td>{i["rule"]}</td>'
                html += f'<td>{str(i["sheet"])[:20]}</td><td>{str(i["column"])[:12]}</td><td>{i["row"]}</td>'
                html += f'<td>{str(i["value"])[:15]}</td><td>{i["message"]}</td></tr>'
            html += '</table></div>'
        display(HTML(html))

def display_downloads(files):
    with out_downloads:
        clear_output(wait=True)
        for fp in files:
            name = os.path.basename(fp)
            with open(fp, 'rb') as f: b64 = base64.b64encode(f.read()).decode()
            display(HTML(f'<a href="data:application/octet-stream;base64,{b64}" download="{name}" '
                         f'style="display:inline-block; padding:8px 15px; background:#4caf50; color:white; '
                         f'text-decoration:none; border-radius:4px; margin:5px 0;">Download {name}</a>'))

upload1.observe(lambda c: save_upload(upload1, 'boundary_file'), names='value')
upload2.observe(lambda c: save_upload(upload2, 'facility_file'), names='value')
btn_validate.on_click(on_validate)
btn_clear.on_click(on_clear)

display(widgets.VBox([
    widgets.HTML('<h3>Upload & Validate</h3>'),
    widgets.HTML('<b>Boundary File:</b>'), upload1,
    widgets.HTML('<b>Facility File:</b>'), upload2,
    out_status,
    widgets.HBox([btn_validate, btn_clear]),
    out_results, out_downloads
]))